# Experimento Doc2Vec

O objetivo aqui é usar doc2vec para classificação multi-classe no mini-dataset de atos/classes gerado após teste no ezTag.

Fontes principais:
- https://towardsdatascience.com/implementing-multi-class-text-classification-with-doc2vec-df7c3812824d
- https://github.com/UnB-KnEDLe/experiments/blob/master/members/matheus/BioC_XML_to_conll.ipynb

### Import de packages

In [1]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import utils
import csv
from tqdm import tqdm
import multiprocessing
import nltk
from nltk.corpus import stopwords

### Mini-Dataset de atos
- Instanciação em memória
- Retirada de classes com apenas 1 instância

In [2]:
import pandas as pd

df = pd.read_csv('seg_textos/atos_segs.csv')
df

,tipo_ato,ato
0,nomeacao,NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o ...
1,exoneracao,"EXONERAR, a pedido, ANNA LUIZA BARCIA HALLEY d..."
2,nomeacao,NOMEAR SANDRA TURCATO JORGE TOLENTINO para exe...
3,nomeacao,"NOMEAR RAIMUNDO DA COSTA SANTOS NETO, Procurad..."
4,ato_tornado_sem_efeito,TORNAR SEM EFEITO no Decreto de 02 de julho de...
...,...,...
268,substituicao,DESIGNAR\nrespectivamente TEREZA CRISTINA DE A...
269,substituicao,"Designar FLÁVIO DE ARAÚJO ALMEIDA, matrícula n..."
270,substituicao,"DESIGNAR OTAMÁ DANTAS BARRETO, matrícula\n159...."
271,retificacao,"RETIFICAR o Sexto Termo\nAditivo ao Contrato, ..."


In [3]:
df.tipo_ato.value_counts()

aposentadoria             69
nomeacao                  53
exoneracao                49
retificacao               38
substituicao              37
ato_tornado_sem_efeito    16
cessao                     9
reversao                   1
abono_de_permanência       1
Name: tipo_ato, dtype: int64

In [4]:
df = df[df.tipo_ato != 'abono_de_permanência']
df = df[df.tipo_ato != 'reversao']
df.tipo_ato.value_counts()

aposentadoria             69
nomeacao                  53
exoneracao                49
retificacao               38
substituicao              37
ato_tornado_sem_efeito    16
cessao                     9
Name: tipo_ato, dtype: int64

### Criação de listas de atos e tipos de ato, em ordem

In [6]:
col_names = ['tipo_ato', 'ato']
dt = pd.read_csv('seg_textos/atos_segs.csv', names=col_names)
atos = dt.ato.to_list()
tipos = dt.tipo_ato.to_list()
atos.remove('ato')
tipos.remove('tipo_ato')

### Retira \n das instâncias de atos

In [77]:
import re

at_contiguos = []
for ato in atos:
    at_contiguos.append(re.sub('\n', ' ', ato))
#at_contiguos

### Cria lista de listas de referência

infos é uma lista que contém duas outras listas:
- tipos
- at_contiguos

Ela servirá de referência para mapear instâncias

In [8]:
infos = []
infos.append(tipos)
infos.append(at_contiguos)
infos

infos[0][1]

'exoneracao'

### Função para tokenização

In [9]:
tqdm.pandas(desc="progress-bar")
# Function for tokenizing
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [10]:
tokenize_text(at_contiguos[0])

['nomear',
 'letícia',
 'matos',
 'magalhães',
 'para',
 'exercer',
 'cargo',
 'em',
 'comissão',
 'símbolo',
 'cc-04',
 'código',
 'sigrh',
 'b0001615',
 'de',
 'assessor',
 'técnico',
 'da',
 'chefia',
 'de',
 'gabinete',
 'do',
 'gabinete',
 'do',
 'governador']

### Mapear labels

In [11]:
possible_labels = dt.tipo_ato.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [12]:
df['label'] = df.tipo_ato.replace(label_dict)
df

,tipo_ato,ato,label
0,nomeacao,NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o ...,1
1,exoneracao,"EXONERAR, a pedido, ANNA LUIZA BARCIA HALLEY d...",2
2,nomeacao,NOMEAR SANDRA TURCATO JORGE TOLENTINO para exe...,1
3,nomeacao,"NOMEAR RAIMUNDO DA COSTA SANTOS NETO, Procurad...",1
4,ato_tornado_sem_efeito,TORNAR SEM EFEITO no Decreto de 02 de julho de...,3
...,...,...,...
268,substituicao,DESIGNAR\nrespectivamente TEREZA CRISTINA DE A...,5
269,substituicao,"Designar FLÁVIO DE ARAÚJO ALMEIDA, matrícula n...",5
270,substituicao,"DESIGNAR OTAMÁ DANTAS BARRETO, matrícula\n159....",5
271,retificacao,"RETIFICAR o Sexto Termo\nAditivo ao Contrato, ...",4


### Divisão treino e teste para fins de indexação

Vamos usar os valores de X_train, X_test, y_train, y_test para popular listas de treino e teste que serão posteriormente usadas em objetos TaggedDocument para usar com doc2vec

In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.25, 
                                                  random_state=17, 
                                                  stratify=df.label.values)

In [51]:
X_train, X_test, y_train, y_test

(array([  4, 192,  70, 132,  83, 186,  91, 195,  82,  48, 256,  87,  26,
         69, 136, 204,  22, 179,  27, 216,  72, 261,   0,  80, 169, 157,
        246, 189, 207, 110,  57, 240, 137,  81, 102, 115, 241, 103,  10,
         19, 190,  29,  16, 234, 184, 205, 151, 180,  71,  86, 231,   1,
        257,  45, 174, 188, 260, 218, 214, 202, 248,  21, 139, 173,   5,
        165,  32,  58, 167, 177,  76,  28,   6,  49, 220, 134,  56,   7,
        131, 187, 237, 166,  17, 101, 225, 228, 269, 119,  94, 142, 250,
        217, 107,  41, 252,  78,  55, 135, 270,  59, 201, 211,  34, 153,
         13,  25, 209,  12, 244,  23,  73,  60, 268,  30,  18,  36,  42,
        224, 196, 128,  92, 242, 112,  66, 213,  84, 122, 170,  50, 208,
         43,  46,  53, 222,  37,  24,  74,  54, 105, 109, 159, 100, 198,
        164, 148, 254, 121, 194,  61, 243, 124, 158, 138, 182, 266, 253,
        221, 162, 152, 230, 263, 133,  38,  97,  88,  65, 147,  77,   8,
        145, 108, 212, 168, 203, 183, 239,   3, 226

### Conversão numpy.int64 para int nativo da linguagem

In [26]:
print(type(X_train[0]))
a = X_train[0].item()
print(a)
print(type(a))

<class 'numpy.int64'>
4
<class 'int'>


In [33]:
index_doc = X_train[0].item()
print(index_doc)
print(type(index_doc))
doc = at_contiguos[index_doc]
print(doc)
print(type(doc))
tag = y_train[0].item()
print(tag)
print(type(tag))

4
<class 'int'>
TORNAR SEM EFEITO no Decreto de 02 de julho de 2020, publicado no DODF nº 124, de 03 de julho de 2020, página 18, o ato que nomeou ERNANY SANTOS DE ALMEIDA para exercer o Cargo Público de Natureza Especial, Símbolo CPE-02, código SIGRH 65260137, de Subsecretário, da Subsecretaria de Planejamento, Acompanhamento e Avaliação, da Secretaria de Estado de Educação do Distrito Federal.
<class 'str'>
3
<class 'int'>


In [78]:
# Initializing the variables
train_documents = []
test_documents = []

j = 0
for i in X_train:
    index_doc = i.item()
    doc = at_contiguos[index_doc]
    tag = y_train[j].item()
    train_documents.append(TaggedDocument(words=tokenize_text(doc), tags=[tag]))
    j+=1

j = 0
for i in X_test:
    index_doc = i.item()
    doc = at_contiguos[index_doc]
    tag = y_test[j].item()
    test_documents.append(TaggedDocument(words=tokenize_text(doc), tags=[tag]))
    j+=1

In [68]:
train_documents[7]

TaggedDocument(words=['designar', 'servidora', 'rosângela', 'davi', 'de', 'carvalho', 'matrícula', 'nº', '387.606', 'técnico', 'em', 'políticas', 'públicas', 'gestão', 'governamental', 'para', 'substituir', 'sem', 'acumular', 'vencimentos', 'sem', 'prejuízos', 'de', 'suas', 'atribuições', 'servidora', 'eliane', 'delfino', 'matrícula', 'nº', '0166951-1', 'gerente', 'da', 'gerência', 'de', 'pessoas', 'da', 'coordenação', 'de', 'administração', 'geral', 'da', 'administração', 'regional', 'do', 'paranoá', 'símbolo', 'cpc-08', 'no', 'período', 'de', '16', '25', 'de', 'novembro', 'de', '2020', 'referente', 'ao', 'período', 'de', 'férias', 'da', 'titular'], tags=[5])

### Modelo doc2vec
- Instanciação
- Divisão treino e teste para uso no modelo em si
- Utilização do modelo com LogisticRegression

In [69]:
cores = multiprocessing.cpu_count()

model_dbow = Doc2Vec(dm=1, vector_size=100, negative=5, hs=0, min_count=2, sample = 0, workers=cores, alpha=0.025, min_alpha=0.001)
model_dbow.build_vocab([x for x in tqdm(train_documents)])
train_documents  = utils.shuffle(train_documents)
model_dbow.train(train_documents,total_examples=len(train_documents), epochs=30)

def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags, model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, feature_vectors

model_dbow.save('./actModel.d2v')

100%|██████████| 203/203 [00:00<00:00, 393458.28it/s]


In [70]:
y_train_model, X_train_model = vector_for_learning(model_dbow, train_documents)
y_test_model, X_test_model = vector_for_learning(model_dbow, test_documents)

In [76]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train_model, y_train_model)
y_pred = logreg.predict(X_test_model)
print('Testing accuracy for movie plots%s' % accuracy_score(y_test_model, y_pred))
print('Testing F1 score for movie plots: {}'.format(f1_score(y_test_model, y_pred, average='weighted')))

Testing accuracy for movie plots0.8088235294117647
Testing F1 score for movie plots: 0.8107609710550887


/home/mstauffer/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/mstauffer/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
